# Pix2Pix in Tensorflow

This tutorials walks through an implementation of Pix2Pix as described in [Image-to-Image Translation with Conditional Adversarial Networks](https://arxiv.org/abs/1611.07004).

This specific implementation is designed to 'remaster' black-and-white frames from films with 4:3 aspect ratio into full-color and 16:9 aspect ratio frames. 

To learn more about the Pix2Pix framework, and the images that can be generated using this framework, see my [Medium post](https://medium.com/p/f4d551fa0503).

This notebook requires the additional helper.py file, which can be obtained [here]()

In [ ]:
#Import the libraries we will need.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import os
import scipy.misc
import scipy
from PIL import Image
from glob import glob
import os
from helper import *
%matplotlib inline

#Size of image frames
height = 144
width = 256

## Defining the Adversarial Networks

### Generator Network

In [ ]:
def generator(c):
    with tf.variable_scope('generator'):
        #Encoder
        enc0 = slim.conv2d(c,64,[3,3],padding="SAME",
            biases_initializer=None,activation_fn=lrelu,
            weights_initializer=initializer)
        enc0 = tf.space_to_depth(enc0,2)
        
        enc1 = slim.conv2d(enc0,128,[3,3],padding="SAME",
            activation_fn=lrelu,normalizer_fn=slim.batch_norm,
            weights_initializer=initializer)
        enc1 = tf.space_to_depth(enc1,2)

        enc2 = slim.conv2d(enc1,128,[3,3],padding="SAME",
            normalizer_fn=slim.batch_norm,activation_fn=lrelu,
            weights_initializer=initializer)
        enc2 = tf.space_to_depth(enc2,2)

        enc3 = slim.conv2d(enc2,256,[3,3],padding="SAME",
            normalizer_fn=slim.batch_norm,activation_fn=lrelu,
            weights_initializer=initializer)
        enc3 = tf.space_to_depth(enc3,2)
        
        #Decoder
        gen0 = slim.conv2d(
            enc3,num_outputs=256,kernel_size=[3,3],
            padding="SAME",normalizer_fn=slim.batch_norm,
            activation_fn=tf.nn.elu, weights_initializer=initializer)
        gen0 = tf.depth_to_space(gen0,2)

        gen1 = slim.conv2d(
            tf.concat(3,[gen0,enc2]),num_outputs=256,kernel_size=[3,3],
            padding="SAME",normalizer_fn=slim.batch_norm,
            activation_fn=tf.nn.elu,weights_initializer=initializer)
        gen1 = tf.depth_to_space(gen1,2)

        gen2 = slim.conv2d(
            tf.concat(3,[gen1,enc1]),num_outputs=128,kernel_size=[3,3],
            padding="SAME",normalizer_fn=slim.batch_norm,
            activation_fn=tf.nn.elu,weights_initializer=initializer)
        gen2 = tf.depth_to_space(gen2,2)

        gen3 = slim.conv2d(
            tf.concat(3,[gen2,enc0]),num_outputs=128,kernel_size=[3,3],
            padding="SAME",normalizer_fn=slim.batch_norm,
            activation_fn=tf.nn.elu, weights_initializer=initializer)
        gen3 = tf.depth_to_space(gen3,2)
        
        g_out = slim.conv2d(
            gen3,num_outputs=3,kernel_size=[1,1],padding="SAME",
            biases_initializer=None,activation_fn=tf.nn.tanh,
            weights_initializer=initializer)
        return g_out

### Discriminator Network

In [ ]:
def discriminator(bottom, reuse=False):
    with tf.variable_scope('discriminator'):
        filters = [32,64,128,128]
        
        #Programatically define layers
        for i in range(len(filters)):
            if i == 0:
                layer = slim.conv2d(bottom,filters[i],[3,3],padding="SAME",scope='d'+str(i),
                    biases_initializer=None,activation_fn=lrelu,stride=[2,2],
                    reuse=reuse,weights_initializer=initializer)
            else:
                layer = slim.conv2d(bottom,filters[i],[3,3],padding="SAME",scope='d'+str(i),
                    normalizer_fn=slim.batch_norm,activation_fn=lrelu,stride=[2,2],
                    reuse=reuse,weights_initializer=initializer)
            bottom = layer

        dis_full = slim.fully_connected(slim.flatten(bottom),1024,activation_fn=lrelu,scope='dl',
            reuse=reuse, weights_initializer=initializer)

        d_out = slim.fully_connected(dis_full,1,activation_fn=tf.nn.sigmoid,scope='do',
            reuse=reuse, weights_initializer=initializer)
        return d_out

### Connecting them together

In [ ]:
tf.reset_default_graph()

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#
condition_in = tf.placeholder(shape=[None,height,width,3],dtype=tf.float32)
real_in = tf.placeholder(shape=[None,height,width,3],dtype=tf.float32) #Real images

Gx = generator(condition_in) #Generates images from random z vectors
Dx = discriminator(real_in) #Produces probabilities for real images
Dg = discriminator(Gx,reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
#For generator we use traditional GAN objective as well as L1 loss
g_loss = -tf.reduce_mean(tf.log(Dg)) + 100*tf.reduce_mean(tf.abs(Gx - real_in)) #This optimizes the generator.

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.002,beta1=0.5)
d_grads = trainerD.compute_gradients(d_loss,slim.get_variables(scope='discriminator'))
g_grads = trainerG.compute_gradients(g_loss, slim.get_variables(scope='generator'))

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)

## Training the network
Now that we have fully defined our network, it is time to train it!

In [ ]:
batch_size = 4 #Size of image batch to apply at each iteration.
iterations = 500000 #Total number of iterations to use.
subset_size = 5000 #How many images to load at a time, will vary depending on available resources
frame_directory = './frames' #Directory where training images are located
sample_directory = './samples' #Directory to save sample images from generator in.
model_directory = './model' #Directory to save trained model to.
sample_frequency = 200 #How often to generate sample gif of translated images.
save_frequency = 5000 #How often to save model.
load_model = False #Whether to load the model or begin training from scratch.

In [ ]:
subset = 0
dataS = sorted(glob(os.path.join(frame_directory, "*.png")))
total_subsets = len(dataS)/subset_size
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(init)
    if load_model == True: 
        ckpt = tf.train.get_checkpoint_state(model_directory)
        saver.restore(sess,ckpt.model_checkpoint_path)

    imagesY,imagesX = loadImages(dataS[0:subset_size],False, np.random.randint(0,2)) #Load a subset of images
    print "Loaded subset " + str(subset)
    draw = range(len(imagesX))
    for i in range(iterations):
        if i % (subset_size/batch_size) != 0 or i == 0:
            batch_index = np.random.choice(draw,size=batch_size,replace=False)
        else:
            subset = np.random.randint(0,total_subsets+1)
            imagesY,imagesX = loadImages(dataS[subset*subset_size:(subset+1)*subset_size],False, np.random.randint(0,2))
            print "Loaded subset " + str(subset)
            draw = range(len(imagesX))
            batch_index = np.random.choice(draw,size=batch_size,replace=False)
        
        ys = (np.reshape(imagesY[batch_index],[batch_size,height,width,3]) - 0.5) * 2.0 #Transform to be between -1 and 1
        xs = (np.reshape(imagesX[batch_index],[batch_size,height,width,3]) - 0.5) * 2.0
        _,dLoss = sess.run([update_D,d_loss],feed_dict={real_in:ys,condition_in:xs}) #Update the discriminator
        _,gLoss = sess.run([update_G,g_loss],feed_dict={real_in:ys,condition_in:xs}) #Update the generator
        if i % 200 == 0:
            print "Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss)
            start_point = np.random.randint(0,len(imagesX)-32)
            xs = (np.reshape(imagesX[start_point:start_point+32],[32,height,width,3]) - 0.5) * 2.0
            ys = (np.reshape(imagesY[start_point:start_point+32],[32,height,width,3]) - 0.5) * 2.0
            sample_G = sess.run(Gx,feed_dict={condition_in:xs}) #Use new z to get sample images from generator.
            allS = np.concatenate([xs,sample_G,ys],axis=1)
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            #Save sample generator images for viewing training progress.
            make_gif(allS,'./'+sample_directory+'/a_vid'+str(i)+'.gif',
                duration=len(allS)*0.2,true_image=False)
        if i % 5000 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
            print "Saved Model"

## Using a trained network
Once we have a trained model saved, we may want to use it to generate new images, and explore the representation it has learned.

In [ ]:
test_directory = './test_frames' #Directory to load test frames from
subset_size = 5000
batch_size = 60 # Size of image batch to apply at each iteration. Will depend of available resources.
sample_directory = './test_samples' #Directory to save sample images from generator in.
model_directory = './model' #Directory to save trained model to.
load_model = True #Whether to load a saved model.

In [ ]:
dataS = sorted(glob(os.path.join(test_directory, "*.png")))
subset = 0
total_subsets = len(dataS)/subset_size
iterations = subset_size / batch_size #Total number of iterations to use.


if not os.path.exists(sample_directory):
    os.makedirs(sample_directory)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(init)
    if load_model == True: 
        ckpt = tf.train.get_checkpoint_state(model_directory)
        saver.restore(sess,ckpt.model_checkpoint_path)
        for s in range(total_subsets):
            generated_frames = []
            _,imagesX = loadImages(dataS[s*subset_size:s*subset_size+subset_size],False, False) #Load a subset of images
            for i in range(iterations):
                start_point = i * batch_size
                xs = (np.reshape(imagesX[start_point:start_point+batch_size],[batch_size,height,width,3]) - 0.5) * 2.0
                sample_G = sess.run(Gx,feed_dict={condition_in:xs}) #Use new z to get sample images from generator.    
                #allS = np.concatenate([xs,sample_G],axis=2)
                generated_frames.append(sample_G)
            generated_frames = np.vstack(generated_frames)
            for i in range(len(generated_frames)):
                im = Image.fromarray(((generated_frames[i]/2.0 + 0.5) * 256).astype('uint8'))
                im.save('./'+sample_directory+'/frame'+str(s*subset_size + i)+'.png')  
            #make_gif(generated_frames,'./'+sample_directory+'/a_vid'+str(i)+'.gif',
            #    duration=len(generated_frames)/10.0,true_image=False)